# Map Reduce

Eerst maken we een aparte directory aan voor alles wat we voor deze notebook gaan gebruiken in hdfs. Dit om conflicten of het overschrijven van gegevens te vermijden.


In [1]:
from hdfs import InsecureClient

map = 'MapReduce'

client = InsecureClient('http://localhost:9870', user='bigdata')

if client.status(map, strict=False) is None:
    client.makedirs(map)
else:
    # do some cleaning in case anything else than *.txt is present
    for f in client.list(map):
        client.delete(map + '/' + f, recursive=True)

client.upload(map, 'input.txt')
client.upload(map, 'titanic.csv')
with client.read(map + '/input.txt') as reader:
    content = reader.read()
    print(content.decode('utf-8'))

﻿Hello World,
hello world,
hello world,

Dit is een voorbeeld file om het Wordcount voorbeeld te testen !


## Wat is MapReduce

MapReduce is een programmeermodel om eenvoudig distributed data te verwerken.
Het is belangrijk om te realiseren dat de programma's die je hier schrijft een parallel uitgevoerd worden op verschillende stukjes data (De map-fase) om daarna in de reduce-fase tot een finale output teruggebracht te worden.
Er gebeuren 5 stappen bij het uitvoeren van een MapReduce programma
* Bepalen op welke nodes de code uitgevoerd wordt (wordt door YARN gedaan afhankelijk van de locatie van de blocks)
* Uitvoeren van de Map-code (Geschreven door de developer naar eigen wens)
* Shuffle, ouput van de map-fase doorsturen naar andere nodes die de resultaten gaan reduceren (Automatisch)
* uitvoeren van de Reduce-code (Geschreven door de developer naar eigen wens)
* Combineren van de reduce output tot 1 gehele/finale output (Automatisch)

Uit bovenstaand stappenplan is het duidelijk dat er twee zaken moeten geimplementeerd worden bij het schrijven van een MapReduce toepassing.
Echter zullen we eerst een aantal voorbeelden bestuderen met reeds bestaande implementaties om zo meer vertrouwd te geraken met de flow van MapReduce.

## Voorbeelden van bestaande applicaties

Reeds een aantal default MapReduce applications zijn mee geinstalleerd met Hadoop.
De jar die deze toepassingen bevat kan gevonden worden in hadoop/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.0.jar.
Wanneer je deze jar uitvoert met onderstaande commando krijg je een lijst met de beschikbare applicaties

In [2]:
!hadoop jar hadoop-mapreduce-examples-3.3.6.jar

An example program must be given as the first argument.
Valid program names are:
  aggregatewordcount: An Aggregate based map/reduce program that counts the words in the input files.
  aggregatewordhist: An Aggregate based map/reduce program that computes the histogram of the words in the input files.
  bbp: A map/reduce program that uses Bailey-Borwein-Plouffe to compute exact digits of Pi.
  dbcount: An example job that count the pageview counts from a database.
  distbbp: A map/reduce program that uses a BBP-type formula to compute exact bits of Pi.
  grep: A map/reduce program that counts the matches of a regex in the input.
  join: A job that effects a join over sorted, equally partitioned datasets
  multifilewc: A job that counts words from several files.
  pentomino: A map/reduce tile laying program to find solutions to pentomino problems.
  pi: A map/reduce program that estimates Pi using a quasi-Monte Carlo method.
  randomtextwriter: A map/reduce program that writes 10GB of r

In deze notebook gaan we vooral focussen op het typische probleem van wordcount.
Dit is een toepassing dat gaat tellen hoe vaak elk woord voorkomt in een bepaalde tekst.
Om meer informatie over deze toepassing te krijgen kan je gebruik maken van het volgende commando:

In [3]:
!hadoop jar hadoop-mapreduce-examples-3.3.6.jar wordcount

Usage: wordcount <in> [<in>...] <out>


In de eerste cell hebben we het input.txt bestand geupload. Of dit correct gebeurd is kan je controleren op de [file explorer van het hdfs](http://localhost:9870/explorer.html#/user/bigdata)

Met onderstaande commando kan nu het precompiled word count example uitgevoerd worden

In [4]:
!hadoop jar hadoop-mapreduce-examples-3.3.6.jar wordcount /user/bigdata/MapReduce/input.txt /user/bigdata/MapReduce/output

2025-02-27 08:56:34,619 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at resourcemanager/172.18.0.7:8032
2025-02-27 08:56:34,814 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.18.0.4:10200
2025-02-27 08:56:35,106 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1740574888337_0010
2025-02-27 08:56:35,469 INFO input.FileInputFormat: Total input files to process : 1
2025-02-27 08:56:35,596 INFO mapreduce.JobSubmitter: number of splits:1
2025-02-27 08:56:35,749 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1740574888337_0010
2025-02-27 08:56:35,749 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-02-27 08:56:35,902 INFO conf.Configuration: resource-types.xml not found
2025-02-27 08:56:35,902 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
2025-02-27 08:56:36,317 INFO impl.YarnClientImpl: Submitted application applic

Na het uitvoeren van het wordcount applicatie is er op het HDFS een extra folder toegevoegd met als naam **output**.
De plaats waar de output bewaard wordt is opgegeven in het bovenstaande commando en kan bekeken worden via de file-explorer van het hdfs.
Deze folder bevat de volgende files:
* Een _SUCCESS file dat leeg is. Deze file wordt gebruikt om aan te geven dat de MapReduce applicatie die resulteerde in de output goed afgerond was.
* Een part-r-00000 file dat de output bevat. Indien de output te groot is kan het zijn dat deze file in meerdere files gesplitst wordt. Hier is dit niet het geval en bevat deze file alle ouput. De output van het bovenstaande commando zijn key-value paren waar de keys de verschillende woorden zijn en de values hoeveel keer elk woord voorkomt.

De output van het commando kan dan met onderstaande code uitgelezen worden:

In [5]:
with client.read(map + '/output/part-r-00000') as reader:
    content = reader.read()
    print(content.decode('utf-8'))

!	1
Dit	1
Hello	1
Wordcount	1
World,	1
een	1
file	1
hello	2
het	1
is	1
om	1
te	1
testen	1
voorbeeld	2
world,	2



## Logs/process monitoring

Bovestaande wordcount applicatie wordt uitgevoerd op YARN op de verschillende nodes van de cluster. Dit houdt in dat YARN gebruikt kan worden om de status van de applicatie op te volgen. Hiervoork kan je server naar [de webpagina van Yarn](http://localhost:8088). Na het klikken op de juiste status krijg je de correcte applicatie te zien. Hier kan je ook de logs bekijken door op logs te klikken.

**LET OP: Doordat we in de webbrowser buiten de docker-containers zitten, kunnen we niet de historyserver hostname gebruiken. Als je een DNS-lookup error krijgt, vervang dan de domeinnaam/hostname door localhost.**

## OEFENING: MAPREDUCE 

Probeer nu op basis van bovenstaande reeds bestaande applicaties de gemiddelde lengte van de woorden in de text te bepalen.

In [7]:
# uit te voeren op commando voor de gemiddelde lengte van de woorden te bekomen
!hadoop jar hadoop-mapreduce-examples-3.3.6.jar wordmean /user/bigdata/MapReduce/input.txt /user/bigdata/MapReduce/output_mean

2025-02-27 09:16:29,750 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at resourcemanager/172.18.0.7:8032
2025-02-27 09:16:29,861 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.18.0.4:10200
2025-02-27 09:16:30,084 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1740574888337_0011
2025-02-27 09:16:30,355 INFO input.FileInputFormat: Total input files to process : 1
2025-02-27 09:16:30,450 INFO mapreduce.JobSubmitter: number of splits:1
2025-02-27 09:16:30,606 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1740574888337_0011
2025-02-27 09:16:30,606 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-02-27 09:16:30,754 INFO conf.Configuration: resource-types.xml not found
2025-02-27 09:16:30,754 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
2025-02-27 09:16:31,021 INFO impl.YarnClientImpl: Submitted application applic

De resource manager houdt ook een overzicht bij van de uitgevoerde applicaties, hun status, runtime en eventuele loggings. Na bovenstaande commando's uit te voeren moet je iets gelijkaardigs zien als in de onderstaande screenshot.

![yarn of mapreduce](images/yarn_001.png)

## Zelf implementeren van MapReduce applicaties

Natuurlijk zijn er veel meer zaken mogelijk om te berekenen met map-reduce toepassingen dan de reeds gecompileerde in hadoop.
Zoals eerder aangehaald valt vooral het coderen van de Map- en Reducestap hierbij op de schouders van de developer.
De standaard programmeertaal van MapReduce is Java en dus ook het grootste deel van de documentatie over MapReduce is geschreven met behulp van Java.
Deze programmas moeten dan gecompileerd worden tot een jar dat geupload kan worden naar de overeenkomstige nodes en daar uitgevoerd.
De api overview van hadoop kan je [hier](https://hadoop.apache.org/docs/stable/api/org/apache/hadoop/mapred/package-summary.html) vinden.

De code voor het wordcount example ziet er als volgt uit:

In [14]:
%%file WordCount.java
// dit schrijft onderstaande cell naar een file in de huidige directory
import java.io.IOException;
import java.util.StringTokenizer;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

// Jullie gaan NOOIT moeten coderen in Java!!!!
public class WordCount {

    // Mapper klasse voor de map-functie te doen - tussen de <> - input-key, intput-value, output-key, output-value
    public static class TokenizerMapper extends Mapper<Object, Text, Text, IntWritable>{
        private final static IntWritable one = new IntWritable(1);
        private Text word = new Text();

        public void map(Object key, Text value, Context context) throws IOException, InterruptedException {
            // splits de input string in tokens (standaard op spatie splitsen)
            StringTokenizer itr = new StringTokenizer(value.toString());
            // itereer over alle woorden
            while (itr.hasMoreTokens()){
                // key is het huidige woord
                word.set(itr.nextToken());
                // emit key,value paar
                context.write(word, one);
            }
        }
    }

    // Reducer klasse voor de reduce-functie te doen - tussen de <> - input-key, intput-value, output-key, output-value
    public static class IntSumReducer extends Reducer<Text, IntWritable, Text, IntWritable>{
        private IntWritable result = new IntWritable();

        public void reduce(Text key, Iterable<IntWritable> values, Context context) throws IOException, InterruptedException {
            int sum = 0;
            for(IntWritable val: values){
                sum += val.get();
            }
            result.set(sum);
            context.write(key, result);
        }
    }

   // configure the MapReduce program
    public static void main(String[] args) throws Exception {
        Configuration conf = new Configuration();
        Job job = Job.getInstance(conf, "word count java");
        job.setJarByClass(WordCount.class);
        // configure mapper
        job.setMapperClass(TokenizerMapper.class);
        // configure combiner (soort van reducer die draait op mapping node voor performantie)
        job.setCombinerClass(IntSumReducer.class);
        // configure reducer
        job.setReducerClass(IntSumReducer.class);
        // set output key-value classes
        job.setOutputKeyClass(Text.class);
        job.setOutputValueClass(IntWritable.class);
        // set input file (first argument passed to the program)
        FileInputFormat.addInputPath(job, new Path(args[0]));
        // set output file  (second argument passed to the program)
        FileOutputFormat.setOutputPath(job, new Path(args[1]));
        // In this case, we wait for completion to get the output/logs and not stop the program to early.
        System.exit(job.waitForCompletion(true) ? 0 : 1);
    }
}

Overwriting WordCount.java


Deze code bevat drie delen:
* De main() functie: verzorgt de configuratie van de uit te voeren taak. Geeft aan wat de Map en Reduce klassen zijn, wat de input is, hoe de output bewaard wordt ,...
* De Map-klasse met de map() functie bevat de code voor de mapping-fase
* De Reduce-klasse met de reduce() functie bevat de code voor de reduce-fase

De laatste twee klassen zijn hier gecodeerd als geneste klassen. Deze hadden ook in aparte files geplaatst kunnen worden.
Nu moet deze code eerst omgezet/gecompileerd worden tot een .jar file. Deze kan dan analoog uitgevoerd worden als hierboven met het voorbeeldcode.
Deze twee stappen kunnen uitgevoerd worden door onderstaande commando's.

In [15]:
# compileren tot .jar
!javac -cp "hadoop-common-3.3.6.jar:hadoop-mapreduce-client-core-3.3.6.jar" -d . WordCount.java
!jar cvf wordcounter.jar *.class

added manifest
adding: WordCount$IntSumReducer.class(in = 1739) (out= 738)(deflated 57%)
adding: WordCount$TokenizerMapper.class(in = 1736) (out= 753)(deflated 56%)
adding: WordCount.class(in = 1496) (out= 818)(deflated 45%)


In [16]:
# execute
!hadoop jar wordcounter.jar WordCount /user/bigdata/MapReduce/input.txt /user/bigdata/MapReduce/output_java

2025-02-27 09:32:05,332 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at resourcemanager/172.18.0.7:8032
2025-02-27 09:32:05,482 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.18.0.4:10200
2025-02-27 09:32:05,663 WARN mapreduce.JobResourceUploader: Hadoop command-line option parsing not performed. Implement the Tool interface and execute your application with ToolRunner to remedy this.
2025-02-27 09:32:05,679 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1740574888337_0012
2025-02-27 09:32:05,953 INFO input.FileInputFormat: Total input files to process : 1
2025-02-27 09:32:06,043 INFO mapreduce.JobSubmitter: number of splits:1
2025-02-27 09:32:06,177 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1740574888337_0012
2025-02-27 09:32:06,177 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-02-27 09:32:06,308 INFO conf.Configuratio

## Implementeren via Python

Hoewel het Hadoop Ecosysteem geprogrammeerd is in Java en het dus er goed mee samenwerkt, is het niet verplicht om Java te gebruiken om MapReduce applicaties te schrijven.
Java krijgt namelijk veel kritiek, vooral doordat er veel code nodig is om eenvoudige zaken te programmeren.
Om andere programmeertalen te gebruiken worden er verscheidene API's aangeboden door Hadoop, namelijk
* Hadoop Streaming
    * Communicatie via stdin/stdout
    * Gebruikt door hadoopy, mrjob, ...
* Hadoop Pipes
    * C++ interface voor Hadoop
    * Communicatie via sockets
    * Gebruikt door pydoop


### Mrjob

De eerste API die we bekijken is MrJobs.
Deze API maakt gebruik van de Hadoop Streaming API.
De voordelen van MrJobs zijn:
* Uitgebreidde documentatie
* Code kan lokaal uitgevoerd worden als test
* Data serialisatie gebeurt automatisch (nadeel van Streaming API)
* Werkt met Amazon Elastic MapReduce en Google Cloud Dataproc

Het grootste nadeel is dat de StreamingAPI niet de volledige kracht heeft van het Hadoop ecosysteem omdat alles omgezet wordt naar strings (jsons)

Installatie van deze package gebeurt als volgt:

In [3]:
%%file wordcount_mrjob.py
from mrjob.job import MRJob # importeer de mr job package

class MRWordCount(MRJob): # maak een mapreduce applicatie aan
    #words = {}
    def mapper(self, _, line): 
        # hier schrijf je de code voor te mappen -> dit wordt lijn per lijn opgeroepen
        # dit kan op verschillende nodes uitgevoerd worden voor verschillende blokken -> dus je gaat niet zomaar alles kunnen optellen hier
        for word in line.split():
            # onderstaande kan je niet doen
            #words[word] += 1
            yield (word, 1) # key-value paar om te emitten
            # yield is een return die de code niet stopt

    def reducer(self, word, counts):
        yield (word, sum(counts))

if __name__ == '__main__':
    MRWordCount.run()

Overwriting wordcount_mrjob.py


Deze code kan uitgevoerd worden door het commando hieronder.
Er zijn twee belangrijke parameters om toe te voegen aan het commando.
* -r: Deze parameter geeft aan welke file uitgelezen wordt. Dit kan een lokale file zijn of een file op het hdfs. Om een hdfs-file aan te spreken gebruik je de volgende structuur: hdfs:///{path to file}
* -o: Deze parameter geeft aan waar de data moet bewaard worden. Als deze parameter ontbreekt wordt het in de stdout geprint. Met de parameter kan je aangeven in welke file (lokaal of hdfs) de output moet bewaard worden.

Meer informatie over hoe mrjob applicaties gestart kunnen worden vind je in [de documentatie](https://mrjob.readthedocs.io/en/latest/guides/quickstart.html#running-your-job-different-ways)

In [5]:
!python3 wordcount_mrjob.py -r hadoop hdfs:///user/bigdata/MapReduce/input.txt -o hdfs:///user/bigdata/MapReduce/output_txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop-3.3.6/bin...
Found hadoop binary: /opt/hadoop-3.3.6/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop-3.3.6...
Found Hadoop streaming jar: /opt/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/wordcount_mrjob.root.20250227.095036.934712
uploading working dir files to hdfs:///user/root/tmp/mrjob/wordcount_mrjob.root.20250227.095036.934712/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/wordcount_mrjob.root.20250227.095036.934712/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar573065300816352834/] [] /tmp/streamjob1686846080262696588.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Connecting to Application History server at historyserver/172.18.0.6:10200
  Connecting to ResourceManager at resourcemanager/1

De bovenstaande manier vereist echter dat je steeds een python-file aanmaakt die dan via commandline gestart wordt.
De reden hiervoor is is dat **de file naar de cluster verstuurd wordt**.
Om de output te bekijken kan het output.txt bestand gedownload en uitgeprint worden als volgt:

In [7]:
from hdfs import InsecureClient

map = 'MapReduce'

client = InsecureClient('http://localhost:9870', user='bigdata')

with client.read(map + '/output_txt/part-00000') as reader:
    content = reader.read()
    print(content.decode('utf-8'))

"!"	1
"Dit"	1
"Hello"	1
"Wordcount"	1
"World,"	1
"een"	1
"file"	1
"hello"	2
"het"	1
"is"	1
"om"	1
"te"	1
"testen"	1
"voorbeeld"	2
"world,"	2



## Oefeningen

Maak de nodige mapreduce applicaties voor de volgende zaken te berekenen (je moet de output niet bewaren in een file):
* De gemiddelde woordlengte
* Het aantal keer dat elk karakter voorkomt
* Het aantal woorden dat begint met elke letter
* Het aantal woorden in de tekst

Als een applicatie crasht, kan het zijn dat ze op de cluster nog een hele tijd actief blijft en het starten van nieuwe applicaties kan tegenhouden.
Met onderstaande commando kan je een bestande applicatie afsluiten/killen

In [ ]:
# killen van een applicatie
yarn application -kill <Application_ID>
# appliation id kan je vinden via de webbrowser van yarn (localhost:8088)

In [16]:
%%file vraag1.py
# vraag 1: gemiddelde woordlengte
from mrjob.job import MRJob

class Vraag1(MRJob): 
    def mapper(self, _, line): 
        #lijn per lijn
        for word in line.split():
            # woord per woord
            yield ("lengte", len(word)) # emit hoe lang elk woord is

    def reducer(self, word, counts):
        aantal = 0
        totaal = 0
        for c in counts:
            aantal += 1
            totaal += c

        if aantal == 0:
            yield ('mean length', 0)
        else:
            yield ('mean length', totaal/aantal)
        

if __name__ == '__main__':
    Vraag1.run()

Overwriting vraag1.py


In [17]:
# test vraag 1
!python3 vraag1.py -r hadoop hdfs:///user/bigdata/MapReduce/input.txt -o hdfs:///user/bigdata/MapReduce/vraag1

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop-3.3.6/bin...
Found hadoop binary: /opt/hadoop-3.3.6/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop-3.3.6...
Found Hadoop streaming jar: /opt/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/vraag1.root.20250227.103811.670509
uploading working dir files to hdfs:///user/root/tmp/mrjob/vraag1.root.20250227.103811.670509/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/vraag1.root.20250227.103811.670509/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar4431078611934267120/] [] /tmp/streamjob7541794485209911823.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Connecting to Application History server at historyserver/172.18.0.6:10200
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Connectin

In [10]:
%%file vraag2.py
# vraag 2: het aantal keer dat elk karakter voorkomt
from mrjob.job import MRJob 

class Vraag2(MRJob):
    def mapper(self, _, line): 
        for character in line:
             yield (character, 1)
         
    def reducer(self, character, counts):
        yield (character, sum(counts))

if __name__ == '__main__':
    Vraag2.run()

Writing vraag2.py


In [11]:
# test vraag 2
!python3 vraag2.py -r hadoop hdfs:///user/bigdata/MapReduce/input.txt -o hdfs:///user/bigdata/MapReduce/vraag2

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop-3.3.6/bin...
Found hadoop binary: /opt/hadoop-3.3.6/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop-3.3.6...
Found Hadoop streaming jar: /opt/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/vraag2.root.20250227.103628.066412
uploading working dir files to hdfs:///user/root/tmp/mrjob/vraag2.root.20250227.103628.066412/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/vraag2.root.20250227.103628.066412/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar3305501996250686879/] [] /tmp/streamjob806393535494912102.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Connecting to Application History server at historyserver/172.18.0.6:10200
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Connecting

In [18]:
%%file vraag3.py
# vraag 3: Het aantal woorden dat begint met elke letter
from mrjob.job import MRJob 

class Vraag3(MRJob): 
    def mapper(self, _, line): 
        for word in line.split():
            if len(word)>0 and word[0].isalpha():
                yield (word[0], 1) 

    def reducer(self, word, counts):
        yield (word, sum(counts)) # dit blijft ongewijzigd hiervoor (gewoon optellen nodig)

if __name__ == '__main__':
    Vraag3.run()

Overwriting vraag3.py


In [19]:
# test vraag 3
!python3 vraag3.py -r hadoop hdfs:///user/bigdata/MapReduce/input.txt -o hdfs:///user/bigdata/MapReduce/vraag3

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop-3.3.6/bin...
Found hadoop binary: /opt/hadoop-3.3.6/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop-3.3.6...
Found Hadoop streaming jar: /opt/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/vraag3.root.20250227.103943.504529
uploading working dir files to hdfs:///user/root/tmp/mrjob/vraag3.root.20250227.103943.504529/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/vraag3.root.20250227.103943.504529/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar7283585460625571/] [] /tmp/streamjob7305549278336765119.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Connecting to Application History server at historyserver/172.18.0.6:10200
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Connecting t

In [14]:
%%file vraag4.py
# vraag 4: Het aantal woorden in de tekst
from mrjob.job import MRJob 

class Vraag4(MRJob): 
    def mapper(self, _, line): 
        for word in line.split():
            yield ("aantal woorden", 1) 

    def reducer(self, word, counts):
        yield (word, sum(counts))

if __name__ == '__main__':
    Vraag4.run()

Writing vraag4.py


In [15]:
# test vraag 4
!python3 vraag4.py -r hadoop hdfs:///user/bigdata/MapReduce/input.txt -o hdfs:///user/bigdata/MapReduce/vraag4

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop-3.3.6/bin...
Found hadoop binary: /opt/hadoop-3.3.6/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop-3.3.6...
Found Hadoop streaming jar: /opt/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/vraag4.root.20250227.103741.474595
uploading working dir files to hdfs:///user/root/tmp/mrjob/vraag4.root.20250227.103741.474595/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/vraag4.root.20250227.103741.474595/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar603351783940513249/] [] /tmp/streamjob8684510251235678188.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Connecting to Application History server at historyserver/172.18.0.6:10200
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Connecting

## Werken met MapReduce voor gestructureerde data

Mapreduce kan ook gebruikt worden om te werken met gestructureerde data (bijvoorbeeld een csv) waar elke rij 1 data-element voorsteld.
Het is hier echter wel belangrijk dat alle data op 1 lijn gecombineerd wordt dus multiline csv's, jsons of xml bestanden kunnen niet verwerkt worden.

Onderstaande code is een voorbeeld voor hoe je een csv kan uitlezen en een aantal statistieken kan berekenen. Hierin leer je vooral:
* Hoe de csv lijn per lijn te verwerken en kolommen te detecteren
* Hoe de header rij eruit filteren
* Hoe meerdere berekeningen op een iterator te doen

We gebruiken hiervoor de titanic.csv file. Let op dat die naam van de passagier hierbij komma's kan bevatten dus dit vereist wat extra aandacht.
Daarnaast gebruiken we de lokale versie van het bestand en niet de geuploadde versie.

We willen de volgende zaken berekenen door middel van 1 map-reduce applicatie:
* Gemiddelde leeftijd
* Percentage overleeft
* Percentage mannelijke passagiers
* Percentage vrouwelijke passagiers die het overleefd hebben: 30% betekend dat 30% van de vrouwelijke passagiers het overleefd hebben

In [20]:
%%file structured.py
from mrjob.job import MRJob 
import csv # om csv-based text om te zetten
from io import StringIO

col_age = 5
col_survived = 1
col_gender = 4

class MR_structured(MRJob): 
    def mapper(self, _, line): 
        # col1, col2, col3, col4, ...

        if line == "PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked": # voeg dit toe om niet je headerrij te verwerken
            return
        
        csv_file = StringIO(line)
        cols = next(csv.reader(csv_file)) # zet de csv-text om naar een lijst met kolommen
        # op deze manier kunnen wordt het volgende ook correct omgezet test,"hallo world, met komma", test3
        # 1 zaak om mee op te letten -> elke lijn van de csv 1 entry bevat -> sommige datasets bevatten tekst met newline characters
            # -> die newline characters verwijder je best voor je het bestand oplaad naar het hdfs

        if len(cols) != 12:
            return # indien er iets zou misgegaan zijn met het parsen/lege rijen zou tegenkomen

        # gemiddelde leeftijd
        if cols[col_age] != '': # vermijd null-waarden problemen
            yield ('age', float(cols[col_age])) # zet de string ook om naar een getal

        if cols[col_survived] != '':
            yield('overleefd', int(cols[col_survived]))
            
        if cols[col_gender] != '':
            yield('geslacht', (cols[col_gender]))
            
        if cols[col_gender] != 'female' and cols[col_survived] != '':
            yield('vrouw_overleefd', int(cols[col_survived])) # wat we hier emitten is de volledige survived kolom voor vrouwen

    def reducer(self, key, counts):
        if key == 'age':
            counts = list(counts)
            aantal = len(counts)
            totaal = sum(counts) # als je het niat omzet naar een list gaat deze lijna een fout geven -> counts is een iterator -> kan niet herstarten

            yield ('mean age', totaal/aantal)

        elif key == 'overleefd' or key == 'vrouw_overleefd':
            counts = list(counts)
            aantal = len(counts)
            totaal = sum(counts)

            yield (key, totaal/aantal*100.0)

        elif key == 'geslacht':
            aantal = 0
            for gender in counts:
                if gender == 'male':
                    aantal_mannen += 1
                aantal += 1
            yield ('percentage mannen', aantal_mannen/aantal*100.0)

if __name__ == '__main__':
    MR_structured.run()

Writing structured.py


In [21]:
# test structured
!python3 structured.py -r hadoop hdfs:///user/bigdata/MapReduce/input.txt -o hdfs:///user/bigdata/MapReduce/structured

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop-3.3.6/bin...
Found hadoop binary: /opt/hadoop-3.3.6/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop-3.3.6...
Found Hadoop streaming jar: /opt/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/structured.root.20250227.105846.025241
uploading working dir files to hdfs:///user/root/tmp/mrjob/structured.root.20250227.105846.025241/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/structured.root.20250227.105846.025241/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar3672087349239166291/] [] /tmp/streamjob3257840917041114031.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
  Connecting to Application History server at historyserver/172.18.0.6:10200
  Connecting to ResourceManager at resourcemanager/172.18.0.5:8032

## Debugging

Indien er een python script voor een map-reduce applicatie een error bevat is het niet altijd eenvoudig om de specifieke foutmelding te vinden. Dit is vooral merkbaar als we een runtime errors.
Indien we in het voorgaande script de beveiligingen rond het casten van empty strings verwijderen. Dan krijgen we foutmeldingen. Indien we ditzelfde nu uitvoeren (maar uitvoeren op de cluster in plaats van lokaal in tegenstelling tot het voorgaande commando), dan krijgen we geen duidelijke foutmelding.

In [27]:
%%file structured_error.py

from mrjob.job import MRJob
import csv
from io import StringIO

col_survived = 1
col_sex = 4
col_age = 5

class MR_Structured(MRJob):
    
    def mapper(self, _, line):
        if line == "PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked":
            return
        
        # line of text to list of columns
        csv_file = StringIO(line)
        cols = next(csv.reader(csv_file))    

        yield ("age", float(cols[col_age]))
        yield ("overleefd", int(cols[col_survived]))
        yield ("geslacht" , cols[col_sex])
        yield ("vrouw_overleefd", int(cols[col_survived]))

    def reducer(self, word, counts):

        if word == "age":
            counts = list(counts)
            total = sum(counts)
            size=len(counts)
            yield (word, total/size)
        elif word == "overleefd" or word == "vrouw_overleefd":
            counts = list(counts)
            total = sum(counts)
            size=len(counts)
            yield (word, total/size*100.0)
        elif word == "geslacht": 
            aantal_mannen = 0
            aantal_vrouwen = 0
            for gender in counts:
                if gender == "male":
                    aantal_mannen += 1
                else:
                    aantal_vrouwen += 1
                    
            yield ("percentage mannen", aantal_mannen/(aantal_mannen + aantal_vrouwen) * 100.0)
        else:
            yield("dummy", next(counts))

if __name__ == '__main__':
    MR_Structured.run()

Overwriting structured_error.py


In [29]:
!python3 structured_error.py titanic.csv
# hiermee krijg je onmiddelijk de foutmelding -> er is geen -o gedefinieerd -> schrijf de output hier
# het kan ook voor te testen handiger zijn om -r hadoop weg te laten en eerst lokaal te runnen
# bij de type A/B evaluaties is het niet de bedoeling om het lokaal te runnen maar in de cluster

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/structured_error.root.20250227.110623.045914
Running step 1 of 1...

Error while reading from /tmp/structured_error.root.20250227.110623.045914/step/000/mapper/00000/input:

Traceback (most recent call last):
  File "/home/bigdata/workspace_groep123/Week_03/structured_error.py", line 51, in <module>
    MR_Structured.run()
  File "/usr/local/lib/python3.10/dist-packages/mrjob/job.py", line 616, in run
    cls().execute()
  File "/usr/local/lib/python3.10/dist-packages/mrjob/job.py", line 687, in execute
    self.run_job()
  File "/usr/local/lib/python3.10/dist-packages/mrjob/job.py", line 636, in run_job
    runner.run()
  File "/usr/local/lib/python3.10/dist-packages/mrjob/runner.py", line 503, in run
    self._run()
  File "/usr/local/lib/python3.10/dist-packages/mrjob/sim.py", line 161, in _run
    self._run_step(step, step_num)
  File "/usr/local/lib/python3.10/d

Na het uitvoeren van bovenstaande script, dan zie je dat de applicatie gestart wordt (map 0% reduce 0%) maar een duideljke foutmelding is er niet (run time exception).
Om de logs te bekijken kan je 
* surfen gaan naar de historyserver die beschikbaar is op de url: localhost:8188
* het volgende command-line commando uitvoeren. Dit haalt alle std-err logs van alle containers op. Zoek naar een python error en je zal vinden wat er misgaat.
* een python script schrijven om de yarn log-server te bevragen

In [ ]:
!yarn logs -applicationId {app id} | sed -n '/LogType:stderr/,/End of LogType:stderr/p'

In [30]:
!yarn logs -applicationId application_1740649645876_0007 | sed -n '/LogType:stderr/,/End of LogType:stderr/p'

2025-02-27 11:08:28,100 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at resourcemanager/172.18.0.5:8032
2025-02-27 11:08:28,232 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.18.0.6:10200
LogType:stderr
LogLastModifiedTime:Thu Feb 27 10:59:26 +0000 2025
LogLength:1731
LogContents:
Feb 27, 2025 10:59:02 AM com.sun.jersey.guice.spi.container.GuiceComponentProviderFactory register
INFO: Registering org.apache.hadoop.mapreduce.v2.app.webapp.JAXBContextResolver as a provider class
Feb 27, 2025 10:59:02 AM com.sun.jersey.guice.spi.container.GuiceComponentProviderFactory register
INFO: Registering org.apache.hadoop.yarn.webapp.GenericExceptionHandler as a provider class
Feb 27, 2025 10:59:02 AM com.sun.jersey.guice.spi.container.GuiceComponentProviderFactory register
INFO: Registering org.apache.hadoop.mapreduce.v2.app.webapp.AMWebServices as a root resource class
Feb 27, 2025 10:59:02 AM com.sun.jersey.server.impl.applicat

## Afsluitende opmerkingen

Het is belangrijk om te beseffen dat deze applicatie op meerdere nodes kan tegelijkertijd uitgevoerd worden. 
Dit heeft als gevolg dat je geen persistente counters kan toevoegen in de reducer om gemiddelden en dergelijke te berekenen.
Een globaal overzicht van het dataframe kan maar **in de mapper** behaald worden.

Daarnaast is er nog een andere stap mogelijk dan mapper of reducer. Dat is **de combiner** stap. Dit is een stap die runt per node en gebruikt kan worden om al wat combinaties te doen zodat er minder data tussen nodes moet verstuurd worden. In grote applicaties/datasets kan dit heel wat internettrafiek besparen wat de werking van de cluster ten goede komt.

Ten derde is het ook mogelijk om meerdere stappen te definieren in de mrjob applicatie. Dit kan door de steps functie te implementeren en daar elke stap in te definieren. Elke stap hierin kan beschikken over een eigen mapper/reducer en combiner functie. Meer informatie vind je [in de documentatie](https://mrjob.readthedocs.io/en/latest/guides/quickstart.html#writing-your-second-job)

## Verdere oefeningen

Gebruik nu de titanic csv om met behulp van een mapreduce applicatie de volgende zaken te berekenen:
* Het aantal unieke plaatsen waar personen aan boord zijn gekomen (embarked kolom)
* Het aantal ontbrekende waarden in de Cabin kolom
* De volgende statistische waarden voor de ticketprijs (Fare) kolom: min, max, mean, std
* De langste naam van een passagier
* Het aantal passagiers per klasse
* Het totaal aantal passagiers op de titanic

In [ ]:
%%file structured_oefening.py
